# Archive my tweets

- badges: true
- categories: [code, twitter]


I want to extract all the tweets I've ever written and convert them to small markdown files so they show up as "posts" on this website.

Posts are organized by date, in the traditional blogging format. But so are tweets, kind of? They're listed chronologically anyway. Maybe I could make one post for each day, and then have all the tweets listed on that page. Each one could have its tweet ID as a header, thus having an internal link. Tweets can link to the tweets that precede them, and maybe even backlink to tweets that follow.

It's not quite block references, but as a way of keeping my second brain under my ownership it should work. And this way if anyone wants to cancel me they'll have a convenient search box and permalinks for it. Even if my account gets deleted, my bad takes can stay up.

In [ ]:
#hide_output
import twint
import nest_asyncio
nest_asyncio.apply()


c = twint.Config()

c.Username = 'deepfates'
tweets = []
c.Store_object = True
c.Store_object_tweets_list = tweets


twint.run.Search(c)

1426186893771874306 2021-08-13 08:20:27 -0600 <deepfates> @FractalOntology hahaha
1426186822523174913 2021-08-13 08:20:10 -0600 <deepfates> My biggest peeve with the post rat community is the same reason I left the new age hippie community: low standards for epistemology lead to cults of personality. and a religious regard for irrational thinking
1426186009591664647 2021-08-13 08:16:56 -0600 <deepfates> @Tjdriii yikes 😬 I'm sorry to hear that. probably don't show her this picture
1426185887868719105 2021-08-13 08:16:27 -0600 <deepfates> @bloobsandnoods 😁
1426185761997656068 2021-08-13 08:15:57 -0600 <deepfates> This is fine. Really! it's just this is called practicing your instrument, not playing music.   Just like how it's not really sex if you're doing it by yourself   https://t.co/vJGLgDbRiM
1426180149029085188 2021-08-13 07:53:39 -0600 <deepfates> yeah babe, I'm almost ready to go. Just let me get my boots on  https://t.co/QByQNB0H0W
1426178857388888073 2021-08-13 07:48:31 -0600 <d

In [7]:
#better save that list to disk before i mess around too much more
import pickle

with open('../assets/tweets.pkl', 'wb') as f:
    pickle.dump(tweets, f)

In [12]:
self_tweets = [t for t in tweets if "@" not in t.tweet]

In [23]:
len(self_tweets)

7057

In [451]:
t = tweets[5]

In [452]:
t.conversation_id, t.datestamp, t.datetime, t.id, t.likes_count, t.link, t.mentions, t.photos, t.quote_url, t.replies_count, t.reply_to, t.retweet, t.retweet_date, t.retweet_id, t.retweets_count, t.source, t.thumbnail, t.timestamp, t.timezone, t.tweet, t.urls, t.user_id, t.user_id_str, t.user_rt, t.user_rt_id, t.username, t.video

('1426180149029085188',
 '2021-08-13',
 '2021-08-13 07:53:39 MDT',
 1426180149029085188,
 55,
 'https://twitter.com/deepfates/status/1426180149029085188',
 [],
 ['https://pbs.twimg.com/media/E8rPSWwX0AYVBW5.jpg'],
 '',
 5,
 [],
 False,
 '',
 '',
 4,
 '',
 'https://pbs.twimg.com/media/E8rPSWwX0AYVBW5.jpg',
 '07:53:39',
 '-0600',
 "yeah babe, I'm almost ready to go. Just let me get my boots on  https://t.co/QByQNB0H0W",
 [],
 3315205122,
 '3315205122',
 '',
 '',
 'deepfates',
 1)

In [453]:
import requests
import shutil

In [454]:
def dl_image(url):
    filename = '../images/' + url.split('/')[-1]
    r = requests.get(url, stream = True)
    if r.status_code == 200:
        r.raw.decode_content = True
        with open(filename,'wb') as f:
            shutil.copyfileobj(r.raw, f)
        return(filename)
    else:
        return(None)
    
def image_template(filename):
    return(f'![image from twitter](/fastpages/{filename[2:]})\n')

    
def get_tweet(t):
    if t.photos == []:
        img_md = ''
    else:
        img_list = [dl_image(url) for url in t.photos]
        img_md = '\n'.join([image_template(o) for o in img_list])

    return(f'''
#### <a href = "{t.link}">*{t.timestamp}*</a>

<font size="5">{t.tweet}</font>

{img_md}

🗨️ {t.replies_count} ♺ {t.retweets_count} 🤍  {t.likes_count}   

---
    ''')

def get_md(tweets, date):
    tweets_text = ''.join(t for t in tweets)
    return(f'''---
title: Tweets for {date}
layout: post
toc: true
comments: false
search_exclude: false
categories: [tweets]
---

{tweets_text}
            ''')

In [455]:
from IPython.display import Markdown

In [456]:
Markdown(get_tweet(t))


#### <a href = "https://twitter.com/deepfates/status/1426180149029085188">*07:53:39*</a>

<font size="5">yeah babe, I'm almost ready to go. Just let me get my boots on  https://t.co/QByQNB0H0W</font>

![image from twitter](/fastpages//images/E8rPSWwX0AYVBW5.jpg)


🗨️ 5 ♺ 4 🤍  55   

---
    

### 1426180149029085188
<a href = "https://twitter.com/deepfates/status/1426180149029085188">*07:53:39*</a>

yeah babe, I'm almost ready to go. Just let me get my boots on  https://t.co/QByQNB0H0W

![image from twitter](../images/E8rPSWwX0AYVBW5.jpg)

🗨️5 ♻️4  ❤️55    


In [428]:
yesterday = '2021-08-12'
y_tweets = [tw for tw in tweets if tw.datestamp == yesterday]
len(y_tweets)

220

In [429]:
y_sorted = sorted(y_tweets, key=lambda x: x.datetime)
# [tweet.tweet for tweet in y_sorted]

Too many replies! Let's limit to just mine for now

In [430]:
y_md = get_md([get_tweet(t) for t in y_sorted if "@" not in t.tweet], yesterday)

In [431]:
# y_md

In [434]:
with open(f'../_posts/tweets/{yesterday}-tweets.md', 'w') as f:
    print(y_md, file=f)

Okay, that'll do for now. It prints a chronological page of tweets for each day. Linking, video and oter people's tweets will have to come later.

I'll wrap that behavior in a function and pass it my tweets and a set of dates when i have tweeted.

In [435]:
def write_day_page(day, tweets):
    tweets = [tw for tw in tweets if tw.datestamp == day]
    sorted_tweets = sorted(tweets, key=lambda x: x.datetime)
    md = get_md([get_tweet(t) for t in sorted_tweets], day)
    with open(f'../_posts/tweets/{day}-tweets.md', 'w') as f:
        print(md, file=f)

In [436]:
self_tweets = [t for t in tweets if "@" not in t.tweet]

In [437]:
len(self_tweets)

7057

In [438]:
len([tw for tw in self_tweets if tw.datestamp == yesterday])

43

In [439]:
write_day_page(yesterday, self_tweets)

In [440]:
days = set([t.datestamp for t in self_tweets])

In [441]:
from tqdm import tqdm

In [442]:
for day in tqdm(days):
    write_day_page(day, self_tweets)

100%|██████████| 480/480 [05:21<00:00,  1.49it/s]


I would also liek to do analysis to see how often I tweet. And make a big list of links. Maybe next time.

For now you can find these secret tweet archives by searching in the [Explore](/fastpages/explore) page